In [2]:
import glob
import os
import numpy as np
from PIL import Image
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import cropLargestRecT
from scanimagetiffio import SITiffIO

parent_folder = "/home/zilong/Desktop/2D2P/Data/183_27072023/"

In [3]:
# read the rotation center from the circlecenter txt file
circlecenterfilename = parent_folder+"DP_exp/circlecenter.txt"
with open(circlecenterfilename, "r") as f:
    # read the last row
    last_line = f.readlines()[-1]
    # assign the x and y coordinates to self.rotx and self.roty
    rotx = float(last_line.split()[0])
    roty = float(last_line.split()[1])

In [4]:
#get all the tiff files under the parent folder with an extension of .tif
tifffiles = glob.glob(parent_folder + "/*.tif")
#remove tifffiles with key words "stack"
tifffiles = [x for x in tifffiles if "stack" not in x]
#sort the tifffiles by the number in the file name
tifffiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

#get all the RElog files under the parent folder begin with 'RE' and with an extension of .txt
RElogfiles = glob.glob(parent_folder + "/RE*.txt")
#remove RElogfiles with key words "stack", "00010" and "00012"
RElogfiles = [x for x in RElogfiles if "stack" not in x]


#pair the tiff files and RElog files together which share the same key word
allfiles = []
for tifffile in tifffiles:
    #extract the key word from the tifffile
    #/home/zilong/Desktop/2D2P/Data/183_25072023/25072023_00005.tif' then extract 00005
    key = tifffile.split("/")[-1].split(".")[0].split("_")[-1]
    #find the RElogfile containing the key word
    RElogfile = [x for x in RElogfiles if key in x][0]
    #pair the tifffile and RElogfile together
    pair = [tifffile, RElogfile]
    #append the pair to allfiles
    allfiles.append(pair)

In [5]:
allfiles = allfiles[2:4]
allfiles


[['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00005.tif',
  '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt'],
 ['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00006.tif',
  '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_164024.00006.txt']]

In [5]:
#get mean frame from the first tiff
S = SITiffIO()
S.open_tiff_file(allfiles[0][0], "r")
S.open_rotary_file(allfiles[0][1])
S.interp_times()  # might take a while...
N = S.get_n_frames()

mean_frame = S.get_frame(1)/N
for i in np.arange(1,N):
    frame_i = S.get_frame(i+1)
    mean_frame += frame_i/N

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 28017 timestamps in it.
Counted 9000 frames


In [6]:
#get mean frame from the first tiff
S = SITiffIO()
S.open_tiff_file(allfiles[1][0], "r")
S.open_rotary_file(allfiles[1][1])
S.interp_times()  # might take a while...
N = S.get_n_frames()

mean_frame2 = S.get_frame(1)/N
for i in np.arange(1,N):
    frame_i = S.get_frame(i+1)
    mean_frame2 += frame_i/N

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_164024.00006.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 33581 timestamps in it.
Counted 9000 frames


In [7]:
histoffset = np.median(mean_frame)-np.median(mean_frame2)
histoffset

730.0945555555497

In [7]:
histoffset = np.int16(730)
histoffset

730

In [9]:
#create a folder named UnrotTiff under the parent folder
UnrotTiffFolder = parent_folder + "UnrotTiff_Histmatch/"
if not os.path.exists(UnrotTiffFolder):
    os.makedirs(UnrotTiffFolder)

#for each pair of tiff file and RElog file, unrotate the tiff file and save it into a folder named UnrotTiff
for ind, pair in enumerate(allfiles):
    tiff_file = pair[0]
    RElog_file = pair[1]
    #geterante the unrotated tiff file name
    unrotated_tiff_file = UnrotTiffFolder+tiff_file.split("/")[-1].split(".")[0] + "_unrot2.tif"
    #print processing file name
    print("Processing file: " + tiff_file.split("/")[-1])
  
    #unrotate the tiff file and save it into the UnrotTiff folder
    S = SITiffIO()
    S.open_tiff_file(tiff_file, "r")
    S.open_tiff_file(unrotated_tiff_file, "w")
    S.open_rotary_file(RElog_file)
    S.interp_times()  # might take a while...
    
    N = S.get_n_frames()
    Alltheta = S.get_all_theta()
    print(Alltheta)
    rotCenter = [rotx, roty]
    print(ind)
    for i in range(N):
        frame_i = S.get_frame(i+1)
        theta_i = Alltheta[i]
        #unrotate the frame
        unrotatedFrame = Image.fromarray(frame_i).rotate(theta_i, center=rotCenter)
        #crop the largest inner rectangle from the unrotated frame
        croppedFrame = cropLargestRecT(unrotatedFrame, rotCenter)
        #convert the PIL image back to int16
        if ind>0:
            croppedFrame = np.array(croppedFrame, dtype=np.int16)+histoffset
        else:
            croppedFrame = np.array(croppedFrame, dtype=np.int16)
        S.write_frame(croppedFrame, i)

Processing file: 27072023_00005.tif
Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 28017 timestamps in it.
[254.19129943847656, 251.73587036132812, 247.87173461914062, 241.7771759033203, 236.6999969482422, 231.66195678710938, 226.77064514160156, 222.1630401611328, 215.91195678710938, 210.09130859375, 204.90652465820312, 199.67282104492188, 195.24130249023438, 193.1967315673828, 192.81521606445312, 193.90109252929688, 196.3369598388672, 197.92173767089844, 197.92173767089844, 205.7771759033203, 207.79238891601562, 208.5065155029297, 208.3402099609375, 207.3815155029297, 205.7576141357422, 204.5347900390625, 203.9282684326172, 203.71304321289062, 203.88912963867188, 204.1434783935547, 204.38804626464844, 204.66195678710938, 204.66195678710938, 204.6521759033203, 204.4956512451172, 203.9771728515625, 20

In [13]:
ind

1

In [1]:
tiff_file

NameError: name 'tiff_file' is not defined

In [10]:
allfiles

[['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00005.tif',
  '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_163105.00005.txt'],
 ['/home/zilong/Desktop/2D2P/Data/183_27072023/27072023_00006.tif',
  '/home/zilong/Desktop/2D2P/Data/183_27072023/REdata_20230727_164024.00006.txt']]